<a href="https://colab.research.google.com/github/NicKylis/letter_recognition/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

print ("hello world!")

hello world!


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hojjatk/mnist-dataset")

print("Path to dataset files:", path)

100%|██████████| 22.0M/22.0M [00:00<00:00, 124MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/hojjatk/mnist-dataset/versions/1


In [3]:
import tarfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
# with tarfile.open("res/caltech-101/101_ObjectCategories.tar.gz", "r:gz") as tar:
#     tar.extractall("res/caltech-101")